# EP

### But :  Réserver plusieurs noeuds d'un cluster et lancer smpirun ep à l'aide de "execo.action.Remote" 


#### Préparation de l'env.

In [1]:
%%bash -s
pip3 install --user execo

In [2]:
import execo
from execo import *
from execo_g5k import *
from execo_engine import *
import os
import sys
import json
import time
import re
from shutil import copy
from subprocess import check_output

Réservation noeuds

In [3]:
# Nom du job
jobname = ' '
# Nombre de noeuds
nodecount = 5
# Temps réservation
walltime = "2:0:0" 

# filters out Nancy's graphene cluster (buggy)
resources_selection = "-p \"cluster != 'graphene'\"" 

# Nancy
site = "nancy"

In [4]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname)
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

New job submitted, jobid is 1563812


On les range

In [5]:
jobid = 1563812 
nodes = get_oar_job_nodes(jobid)
nodes.sort(key=lambda n: n.address)
nodes

[Host('griffon-19.nancy.grid5000.fr'),
 Host('griffon-22.nancy.grid5000.fr'),
 Host('griffon-25.nancy.grid5000.fr'),
 Host('griffon-29.nancy.grid5000.fr'),
 Host('griffon-58.nancy.grid5000.fr')]

In [6]:
force_redeploy = False # set to True to force redeploying the OS on the nodes in the deployment section
environment_dsc_file = './debian9-x64-bigdata-tutorial.yaml' # filename of the kadeploy environment file (YAML)

In [9]:
deployment = Deployment(hosts=nodes, env_file = os.path.abspath(environment_dsc_file),
                        other_options="-r ext4 --no-debug-mode")

deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])

nancy:   format_tmp_part
nancy:   format_swap_part
nancy: End of step Deploy[SetDeploymentEnvUntrusted] after 167s
nancy: Performing a Deploy[BroadcastEnvKascade] step
nancy:   send_environment
nancy:    * Broadcast time: 201s
nancy:   manage_admin_post_install
nancy:   manage_user_post_install
nancy:   check_kernel_files
nancy:   send_key
nancy:   install_bootloader
nancy:   sync
nancy: End of step Deploy[BroadcastEnvKascade] after 297s
nancy: Performing a Deploy[BootNewEnvKexec] step
nancy:   switch_pxe
nancy:   umount_deploy_part
nancy:   mount_deploy_part
nancy:   kexec
nancy:   wait_reboot
nancy: End of step Deploy[BootNewEnvKexec] after 61s
nancy: End of deployment for griffon-[19,22,25,29,58].nancy.grid5000.fr after 525s
nancy: End of deployment on cluster griffon after 525s
nancy: Deployment #D-1a1562bc-59b7-43f6-af20-744e936e6a38 done
nancy: 
nancy: The deployment is successful on nodes
nancy: griffon-19.nancy.grid5000.fr
nancy: griffon-22.nancy.grid5000.fr
nancy: griffon-25.n

nancy: Warning: Permanently added 'nancy.grid5000.fr,172.16.79.101' (ECDSA) to the list of known hosts.
nancy: Connection to nancy.grid5000.fr closed.


In [6]:
# execo.action.Remote(cmd,hosts,connection_params,process_args)
Remote_test = execo.action.Remote(cmd='whoami',hosts=nodes,connection_params=None, process_args=None)

In [7]:
Remote_test.run()

Remote('whoami', [Host('griffon-19.nancy.grid5000.fr'), Host('griffon-22.nancy.grid5000.fr'), Host('griffon-25.nancy.grid5000.fr'), Host('griffon-29.nancy.grid5000.fr'), Host('griffon-58.nancy.grid5000.fr')])

In [26]:
Remote_install = execo.action.Remote(cmd='apt-get install g++ libboost-all-dev cmake cmake-curses-gui && wget http://gforge.inria.fr/frs/download.php/latestfile/8/SimGrid-3.18.tar.gz && tar -xvf SimGrid-3.18.tar.gz && cd SimGrid-3.18 && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -Denable_smpi=on -Denable_documentation=off && make -j 4 && make check && make install -j 4',hosts=nodes,connection_params={'user':'root'})

In [27]:
Remote_install.run().ok

Remote('apt-get install g++ libboost-all-dev cmake cmake-curses-gui && wget http://gforge.inria.fr/frs/download.php/latestfile/8/SimGrid-3.18.tar.gz && tar -xvf SimGrid-3.18.tar.gz && cd SimGrid-3.18 && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -Denable_smpi=on -Denable_documentation=off && make -j 4 && make check && make install -j 4', [Host('griffon-19.nancy.grid5000.fr'), Host('griffon-22.nancy.grid5000.fr'), Host('griffon-25.nancy.grid5000.fr'), Host('griffon-29.nancy.grid5000.fr'), Host('griffon-58.nancy.grid5000.fr')], connection_params={'user': 'root'})

In [47]:
Remote_smpi_env = execo.action.Put(hosts=nodes,local_files=["griffon.xml","griffon_hostfile.txt","ep.c","nas_common.h","nas_common.c","Makefile"],connection_params={'user':'root'})
Remote_smpi_env.run().ok

True

In [49]:
Remote_smpi = execo.action.Remote(cmd ='make && && make run',hosts=nodes,connection_params={'user':'root'})
Remote_smpi.run().ok

True

AttributeError: 'Remote' object has no attribute 'stdout'